In [1]:
%load_ext autoreload
%autoreload 2

import torch
import pandas as pd 
import numpy as np

import sklearn

import matplotlib.pyplot as plt
# from torchtext.legacy import data
import spacy
# import en_core_web_sm

from src.tdde13.data_handling import * #preprocess, preprocess_text, create_adjacency_matrix, create_label_lookup
from src.tdde13.lstm import LSTM
# , exclude=["ner", "parser"]
nlp = spacy.load("en_core_web_sm", exclude=["ner", "parser"])

import pickle

import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [ ]:
# mit reviews arbeiten
# wer könnten mit nlp gearbeitet haben? John?
# 

In [2]:
df = pd.read_csv('datasets/book_reviews/finalv2.csv')

In [4]:
# next(iter(pd.read_csv('datasets/book_reviews/final.csv', chunksize=10)))
df = pd.read_csv('datasets/book_reviews/finalv2.csv')
# df5000 = df[:5000]
df_subset = df[:10000]

load = True
if load:
    with open('objs10000.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
        desc, vocab, word_to_ix = pickle.load(f)
else:
    desc, vocab, word_to_ix = preprocess_text(nlp, df10000.description)
    with open('objs5000.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([desc, vocab, word_to_ix], f)

label_to_idx = create_label_lookup(df_subset)
clean_desc, labels = remove_empty_descriptions(desc, df_subset.genres)

In [5]:
df_subset = df[:10000]
clean_desc, labels = remove_empty_descriptions(desc, df_subset.genres)
padded_desc, max_len = pad_sequence(clean_desc)

In [7]:
idx_train, idx_val, idx_test = split(labels=labels, train_size=0.6, val_size=0.2, test_size=0.2)

In [8]:
padded_desc[idx_train]

array([['Davey', 'feel', 'life', ..., '<pad>', '<pad>', '<pad>'],
       ['blind', 'miserable', 'able', ..., '<pad>', '<pad>', '<pad>'],
       ['New', 'York', 'Timesbestselling', ..., '<pad>', '<pad>',
        '<pad>'],
       ...,
       ['year', 'Annabel', 'girl', ..., '<pad>', '<pad>', '<pad>'],
       ['straight', 'library', 'strange', ..., '<pad>', '<pad>', '<pad>'],
       ['take', 'bite', 'Big', ..., '<pad>', '<pad>', '<pad>']],
      dtype=object)

In [9]:
padded_desc[idx_val]

array([['Lina', 'like', 'year', ..., '<pad>', '<pad>', '<pad>'],
       ['year', 'world', 'bleak', ..., '<pad>', '<pad>', '<pad>'],
       ['Time', 'City', 'build', ..., '<pad>', '<pad>', '<pad>'],
       ...,
       ['eighteen', 'year', 'old', ..., '<pad>', '<pad>', '<pad>'],
       ['friend', 'benefit', 'read', ..., '<pad>', '<pad>', '<pad>'],
       ['Adam', 'kindly', 'leader', ..., '<pad>', '<pad>', '<pad>']],
      dtype=object)

In [10]:
padded_desc[idx_test]

array([['Humphrey', 'Wescott', 'Earl', ..., '<pad>', '<pad>', '<pad>'],
       ['Sadie', 'Turner', 'organize', ..., '<pad>', '<pad>', '<pad>'],
       ['belong', 'wild', 'fantasy', ..., '<pad>', '<pad>', '<pad>'],
       ...,
       ['Clementine', 'DeVore', 'spend', ..., '<pad>', '<pad>', '<pad>'],
       ['Neal', 'Barton', 'want', ..., '<pad>', '<pad>', '<pad>'],
       ['Sally', 'Paul', 'penny', ..., '<pad>', '<pad>', '<pad>']],
      dtype=object)

In [ ]:
tensor = torch.tensor([[3,5,1,2],[3,1,5,3],[7,5,8,3]],dtype=torch.float)
print(tensor)
tensor.apply_(lambda x: (x+0.2))
print(tensor)

In [ ]:
# https://suzyahyah.github.io/pytorch/2019/07/01/DataLoader-Pad-Pack-Sequence.html
'''from torch import nn
from torch.nn.utils.rnn import pad_sequence
x_seq = [[5, 18, 29], [32, 100], [699, 6, 9, 17]]
x_padded = pad_sequence(x_seq, batch_first=True, padding_value=0)'''

In [ ]:
pd.DataFrame(labels).value_counts().plot(kind='bar')

In [ ]:
pd.DataFrame(labels).iloc[idx_train.tolist(), :].value_counts().plot(kind='bar')

In [ ]:
pd.DataFrame(labels).iloc[idx_val.tolist(), :].value_counts().plot(kind='bar')

In [11]:
enc_descriptions = torch.tensor([[word_to_ix[word] for word in d] for d in padded_desc])
enc_labels = torch.tensor([label_to_idx[label] for label in labels]).long()


In [31]:
batch = enc_descriptions[idx_test]
label = enc_labels[idx_test]

In [33]:
# what are the predictions of the MLP
# maybe truncate long sentences
# over/undersampling
# more sophisticated preprocessing
predictions = lstm(batch).argmax(axis=1)

In [43]:
predictions[0]

tensor(6)

In [41]:
predictions.apply_(lambda x: idx_to_label[x])

TypeError: an integer is required (got type str)

In [46]:
from sklearn.metrics import classification_report

print(classification_report(label, predictions.numpy(), target_names=list(label_to_idx.keys())))

                                        precision    recall  f1-score   support

              mystery, thriller, crime       0.36      0.13      0.19        63
                           non-fiction       0.00      0.00      0.00        16
                       comics, graphic       0.50      0.10      0.17        30
                              children       0.00      0.00      0.00         6
                           young-adult       0.34      0.39      0.36       175
history, historical fiction, biography       0.50      0.09      0.15        22
                               romance       0.57      0.61      0.59       246
                               fiction       0.31      0.36      0.34       191
                                poetry       0.00      0.00      0.00         1
                   fantasy, paranormal       0.46      0.53      0.50       240

                              accuracy                           0.43       990
                             macro avg

/home/andreas/anaconda3/envs/tdde13/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:

# what are the predictions of the MLP
# maybe truncate long sentences

prediction_train = lstm(enc_descriptions[idx_train]).argmax(axis=1)
label_train = enc_labels[idx_train]
print(classification_report(label_train, prediction_train.numpy(), target_names=list(label_to_idx.keys())))

                                        precision    recall  f1-score   support

              mystery, thriller, crime       1.00      1.00      1.00       505
                           non-fiction       1.00      1.00      1.00       124
                       comics, graphic       1.00      1.00      1.00       243
                              children       1.00      1.00      1.00        49
                           young-adult       1.00      1.00      1.00      1397
history, historical fiction, biography       1.00      1.00      1.00       180
                               romance       1.00      1.00      1.00      1965
                               fiction       1.00      1.00      1.00      1525
                                poetry       1.00      1.00      1.00        10
                   fantasy, paranormal       1.00      1.00      1.00      1914

                              accuracy                           1.00      7912
                             macro avg

In [40]:
pd.DataFrame(predictions.apply_(lambda x: idx_to_label[x]).numpy()).value_counts().plot(kind='bar')

TypeError: an integer is required (got type str)

In [36]:
label_to_idx

{'mystery, thriller, crime': 0,
 'non-fiction': 1,
 'comics, graphic': 2,
 'children': 3,
 'young-adult': 4,
 'history, historical fiction, biography': 5,
 'romance': 6,
 'fiction': 7,
 'poetry': 8,
 'fantasy, paranormal': 9}

In [38]:
idx_to_label = dict(zip(label_to_idx.values(), label_to_idx.keys()))

In [49]:
padded_desc[2]

array(['fresh', 'fun', 'repackage', 'Wild', 'Cards', 'well',
       'Perfectlaunches', 'romantic', 'new', 'series', 'NY',
       'Timesbestselling', 'author', 'Simone', 'Elkeles', 'get', 'kick',
       'boarding', 'school', 'bad', 'boy', 'Derek', 'Fitzpatrick',
       'choice', 'live', 'ditzy', 'stepmother', 'military', 'dad',
       'deploy', 'thing', 'quickly', 'bad', 'bad', 'find', 'plan',
       'childhood', 'home', 'Illinois', 'Derek', 'counting', 'day',
       'thing', 'need', 'involve', 'family', 'drama', 'Ashtyn', 'Parker',
       'know', 'thing', 'certain', 'people', 'care', 'leave', 'backward',
       'glance', 'old', 'sister', 'come', 'home', 'abandon', 'year',
       'early', 'hot', 'new', 'stepson', 'tow', 'Ashtyn', 'want', 'come',
       'plan', 'finally', 'chance', 'leave', 'require', 'trust', 'Derek',
       'barely', 'know', 'bear', 'break', 'rule', 'willing', 'heart',
       'line', 'try', 'future', 'want', '<pad>', '<pad>', '<pad>',
       '<pad>', '<pad>', '<pad>', 

In [14]:
# 987 X 1: 231sec

from src.tdde13.lstm import LSTM
lstm = LSTM(len(vocab), max_len, use_glove=True)
lstm.train_lstm(padded_desc, labels, vocab, word_to_ix, label_to_idx, idx_train, idx_val, idx_test)


Encode Description
Epoch: 0
Train Loss: 86.7609 Train Acc 0.260 Val Loss 28.4830 Val Acc 0.268
Epoch: 1
Train Loss: 77.9744 Train Acc 0.351 Val Loss 28.2019 Val Acc 0.290
Epoch: 2
Train Loss: 73.2569 Train Acc 0.426 Val Loss 28.2310 Val Acc 0.274
Epoch: 3
Train Loss: 68.6127 Train Acc 0.502 Val Loss 28.0607 Val Acc 0.296
Epoch: 4
Train Loss: 64.5757 Train Acc 0.542 Val Loss 27.8488 Val Acc 0.302
Epoch: 5
Train Loss: 60.7472 Train Acc 0.580 Val Loss 27.9713 Val Acc 0.294
Epoch: 6
Train Loss: 57.0952 Train Acc 0.612 Val Loss 28.1256 Val Acc 0.302
Epoch: 7
Train Loss: 53.7181 Train Acc 0.642 Val Loss 28.1880 Val Acc 0.305
Epoch: 8
Train Loss: 50.5277 Train Acc 0.668 Val Loss 28.4793 Val Acc 0.308
Epoch: 9
Train Loss: 47.7426 Train Acc 0.686 Val Loss 28.7887 Val Acc 0.311
Epoch: 10
Train Loss: 44.9336 Train Acc 0.713 Val Loss 28.9648 Val Acc 0.303
Epoch: 11
Train Loss: 42.6001 Train Acc 0.733 Val Loss 29.3599 Val Acc 0.312
Epoch: 12
Train Loss: 40.5582 Train Acc 0.750 Val Loss 29.6456 Val 